# <a id='kernel-approximation'></a> 6.7. [**Approximation de noyaux**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_7_kernel_approximation.ipynb#kernel-approximation)</br>([_Kernel Approximation_](https://scikit-learn.org/stable/modules/kernel_approximation.html#kernel-approximation))

Ce sous-module contient des fonctions qui approximent les mappings de caractéristiques correspondant à certains noyaux, tels qu'ils sont utilisés par exemple dans les machines à vecteurs de support (voir [**Machines à vecteurs de support** (1.4)](https://scikit-learn.org/stable/modules/svm.html#svm)). Les fonctions de caractéristiques suivantes effectuent des transformations non linéaires de l'entrée, qui peuvent servir de base pour la classification linéaire ou d'autres algorithmes.

L'avantage d'utiliser des mappings de caractéristiques explicites approximatifs par rapport à la [**technique des noyaux**](https://en.wikipedia.org/wiki/Kernel_trick), qui utilise les mappings de caractéristiques de manière implicite, est que les mappings explicites peuvent être mieux adaptés à l'apprentissage en ligne et peuvent réduire considérablement le coût de l'apprentissage avec des ensembles de données très volumineux. Les SVM à noyau standard ne sont pas adaptés aux ensembles de données volumineux, mais en utilisant une approximation du mapping de noyau, il est possible d'utiliser des SVM linéaires beaucoup plus efficaces. En particulier, la combinaison d'approximations de mappings de noyaux avec [**`SGDClassifier`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier) permet d'effectuer un apprentissage non linéaire sur des ensembles de données volumineux.

Comme il n'y a pas eu beaucoup de travaux empiriques utilisant des plongements approximatifs, il est conseillé de comparer les résultats avec des méthodes de noyaux exactes lorsque cela est possible.

**Voir aussi :** [**Régression polynomiale : étendre les modèles linéaires avec des fonctions de base** (1.1)](https://scikit-learn.org/stable/modules/linear_model.html#polynomial-regression) pour une transformation polynomiale exacte.